# Case 2 -- Transient, Single-Model, Freshwater Flow

There is recharge for first stress period, and then zero recharge for second stress period.

Two cases are presented here
1. One model layer
2. Three model layers


In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
import swiapi
import flopy

# path to mf6 shared library
# executables based on mf6 feat-swi branch at: 
#   https://github.com/langevin-usgs/modflow6/tree/feat-swi
mf6exe = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
libmf6 = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/libmf6.dylib"


## Case 2a -- One Model Layer

In [ ]:
#create simple test model
Lx = 10000 # meters
delr, delc = 100., 1.
ncol = int(Lx / delr)
nlay = 1
nrow = 1
top = 50.
botm = -400.
recharge = {
    0: 0.001,
    1: 0.,
}
k = 10.
h0 = 0.
h1 = h0
icelltype = 1

ws = './mymodel'
name = 'mymodel'
sim = flopy.mf6.MFSimulation(
    sim_name=name, 
    sim_ws=ws, 
    exe_name=mf6exe,
    memory_print_option="all"
)
nper = 2
nstp = 100
perlen = 200000.
perioddata = nper * [(perlen, nstp, 1.)]
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=perioddata)
ims = flopy.mf6.ModflowIms(
    sim, 
    print_option="summary",
    linear_acceleration="BICGSTAB",
)
gwf = flopy.mf6.ModflowGwf(
    sim, 
    modelname=name, 
    save_flows=True, 
    newtonoptions="newton",
)
dis = flopy.mf6.ModflowGwfdis(
    gwf, 
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=0.)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    alternative_cell_averaging="harmonic",
    icelltype=icelltype,
    k=k,
)
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    iconvert=1,
    ss=1.e-5, 
    sy=0.2
)
zeta_file = name + '.zta'
swi = flopy.mf6.ModflowGwfswi(gwf, zeta_filerecord=zeta_file, zetastrt=-1.)
cghb = 1. * delr * delc / 10.
ghb = flopy.mf6.ModflowGwfghb(gwf, stress_period_data=[[0, 0, 0, h0, cghb],
                                                       [0, 0, ncol - 1, h1, cghb]])
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge)
budget_file = name + '.bud'
head_file = name + '.hds'
oc = flopy.mf6.ModflowGwfoc(
        gwf,
        budget_filerecord=budget_file,
        head_filerecord=head_file,
        saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
        printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
)
sim.write_simulation()
sim.run_simulation()

In [ ]:
import matplotlib.animation
x = gwf.modelgrid.xcellcenters.flatten()
fpth = pl.Path(ws) / f"{gwf.name}.zta"
zeta_all = flopy.utils.HeadFile(fpth, text="zeta").get_alldata()
head_all = gwf.output.head().get_alldata()
times = gwf.output.head().times
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_aspect(10)
ax.set_xlabel(r'x')
ax.set_ylabel(r'y')
title = ax.set_title(f"Time = {times[0]} days")

# plot persistent items
pxs = flopy.plot.PlotCrossSection(gwf, line={"row":0})

def animate(i):
    global title
    zeta = zeta_all[i].flatten()
    head = head_all[i].flatten()
    ax.fill_between(x, head, zeta, color="cyan")
    # saltwater
    ax.fill_between(x, zeta, botm, color="red")
    title = ax.set_title(f"Time = {times[i]} days")

nframes = zeta_all.shape[0]
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=nframes)
plt.close()

from IPython.display import HTML
HTML(ani.to_jshtml())

# Case 2b -- Three Model Layers

In [ ]:
#create simple test model
Lx = 10000 # meters
delr, delc = 100., 1.
ncol = int(Lx / delr)
nlay = 3
nrow = 1
top = 50.
botm = [-50, -200, -400.]
recharge = {
    0: 0.001,
    1: 0.,
}
k = 10.
h0 = 0.
h1 = h0
icelltype = 1

ws = './mymodel'
name = 'mymodel'
sim = flopy.mf6.MFSimulation(
    sim_name=name, 
    sim_ws=ws, 
    exe_name=mf6exe,
    memory_print_option="all"
)
nper = 2
nstp = 100
perlen = 200000.
perioddata = nper * [(perlen, nstp, 1.)]
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=perioddata)
ims = flopy.mf6.ModflowIms(
    sim, 
    print_option="summary",
    linear_acceleration="BICGSTAB",
)
gwf = flopy.mf6.ModflowGwf(
    sim, 
    modelname=name, 
    save_flows=True, 
    newtonoptions="newton",
)
dis = flopy.mf6.ModflowGwfdis(
    gwf, 
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=0.)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    alternative_cell_averaging="harmonic",
    icelltype=icelltype,
    k=k,
)
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    iconvert=1,
    ss=0., 
    sy=0.2
)
zeta_file = name + '.zta'
zetastrt = [-1, -50, -200]
swi = flopy.mf6.ModflowGwfswi(gwf, zeta_filerecord=zeta_file, zetastrt=zetastrt)
cghb = 1. * delr * delc / 10.
ghb = flopy.mf6.ModflowGwfghb(gwf, stress_period_data=[[0, 0, 0, h0, cghb],
                                                       [0, 0, ncol - 1, h1, cghb]])
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge)
budget_file = name + '.bud'
head_file = name + '.hds'
oc = flopy.mf6.ModflowGwfoc(
        gwf,
        budget_filerecord=budget_file,
        head_filerecord=head_file,
        saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
        printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
)
sim.write_simulation()
sim.run_simulation()

In [ ]:
import matplotlib.animation
x = gwf.modelgrid.xcellcenters.flatten()
fpth = pl.Path(ws) / f"{gwf.name}.zta"
zeta_all = flopy.utils.HeadFile(fpth, text="zeta").get_alldata()
head_all = gwf.output.head().get_alldata()
times = gwf.output.head().times
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_aspect(10)
ax.set_xlabel(r'x')
ax.set_ylabel(r'y')
title = ax.set_title(f"Time = {times[0]} days")

# bot = gwf.dis.botm.array
# top = gwf.dis.top.array
# top = np.vstack((top.reshape((1, 1, 100)), bot[:2]))

# plot persistent items
pxs = flopy.plot.PlotCrossSection(gwf, line={"row":0})
l1, = ax.plot(x, np.zeros(x.shape))
l2, = ax.plot(x, np.zeros(x.shape))
l3, = ax.plot(x, np.zeros(x.shape))

def animate(i):
    global title
    head = head_all[i]
    zeta = zeta_all[i]
    # zeta = np.where(zeta < bot, bot, zeta)
    # zeta = np.where(zeta > top, top, zeta)

    l1.set_ydata(zeta[0].flatten())
    l2.set_ydata(zeta[1].flatten())
    l3.set_ydata(zeta[2].flatten())

    title = ax.set_title(f"Time = {times[i]} days")

nframes = zeta_all.shape[0]
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=nframes)
plt.close()

from IPython.display import HTML
HTML(ani.to_jshtml())